In [1]:
!pip install fuzzywuzzy python-Levenshtein

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/94.9 kB ? eta -:--:--
   ------------------------- -------------- 61.4/94.9 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 94.9/94.9 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   --- ------------------------------------ 0.1/1.5 MB 8.3 MB/s eta 0:00:01
   --------- ------------------------------ 0.3/1.5 MB 4.3 MB/s eta 0:00:01
   ---------------- ----------------------- 0.6/1.5 MB 5.7 MB/s eta 0:00:01
   ----------------------- ---------------- 0.9/1.5 MB 5.1 MB/s eta 0:00:01
   ----------------------------- ---------- 1.1/1.5 MB 5.5 MB/s eta 0:00:01
   ----------------------------------- ---- 1.4/1.5 MB 5.8 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 5.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install pycountry

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB 330.3 kB/s eta 0:00:20
    --------------------------------------- 0.1/6.3 MB 1.2 MB/s eta 0:00:06
   - -------------------------------------- 0.2/6.3 MB 1.8 MB/s eta 0:00:04
   -- ------------------------------------- 0.4/6.3 MB 2.3 MB/s eta 0:00:03
   --- ------------------------------------ 0.6/6.3 MB 2.7 MB/s eta 0:00:03
   ---- ----------------------------------- 0.7/6.3 MB 2.4 MB/s eta 0:00:03
   ------ --------------------------------- 1.0/6.3 MB 3.1 MB/s eta 0:00:02
   -------- ------------------------------- 1.3/6.3 MB 3.5 MB/s eta 0:00:02
   ---------- ----------------------------- 1.6/6.3 MB 3.8 MB/s eta 0:00:02
   ------------ --------------------------- 2.0/6.3 MB 4.1 MB/s eta 0:00:02
   -------------- ------------------------- 2.3/6.3 MB 4.2 MB/s eta 0:00:01
   ------------


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import pycountry
import numpy as np
from fuzzywuzzy import process

# --- PHẦN 1: XỬ LÝ DỮ LIỆU CHUỖI THỜI GIAN (DAILY DATA) ---
- Dành cho bạn làm EDA (Đức Chiến) và bạn làm Modeling (Khải An)

## 1. Đọc và chuẩn hóa cơ bản

In [105]:
url_who = "https://raw.githubusercontent.com/NAizdabezt/who-covid19-dashboard/main/data/raw/WHO-COVID-19-global-daily-data.csv"
df_daily = pd.read_csv(url_who)

- Chuẩn hóa tên cột

In [106]:
df_daily.columns = df_daily.columns.str.strip().str.replace(" ", "_").str.replace("/", "_")

- Xử lý ngày tháng

In [107]:
df_daily['Date_reported'] = pd.to_datetime(df_daily['Date_reported'], errors='coerce')

## 2. Xử lý giá trị âm (biến thành NaN)

In [108]:
cols_metric = ['New_cases', 'Cumulative_cases', 'New_deaths', 'Cumulative_deaths']
df_daily[cols_metric] = df_daily[cols_metric].apply(lambda x: x.mask(x < 0))

## 3. Sắp xếp và Nội suy (Interpolation) để lấp đầy NaN

In [109]:
df_daily = df_daily.sort_values(['Country', 'Date_reported'])

def fill_missing_daily(group):
    # Nội suy tuyến tính cho các chỉ số quan trọng
    group[cols_metric] = group[cols_metric].interpolate(method='linear', limit_direction='both')
    return group

df_daily = df_daily.groupby('Country').apply(fill_missing_daily).reset_index(drop=True)
df_daily[cols_metric] = df_daily[cols_metric].fillna(0) # Điền 0 nếu vẫn còn sót

/tmp/ipython-input-1904105238.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_daily = df_daily.groupby('Country').apply(fill_missing_daily).reset_index(drop=True)


## 4. Tính thêm cột Moving Average (Hỗ trợ team EDA vẽ biểu đồ)

In [110]:
df_daily['New_cases_MA7'] = df_daily.groupby('Country')['New_cases'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())
df_daily['New_deaths_MA7'] = df_daily.groupby('Country')['New_deaths'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())

## 5. Tính tốc độ tăng trưởng (Growth Rate) = (Hôm nay - Hôm qua) / Hôm qua

- Biến này giúp team EDA vẽ biểu đồ tốc độ lây lan

In [111]:
df_daily['Growth_Rate'] = df_daily.groupby('Country')['New_cases'].pct_change() * 100

- Xử lý lỗi chia cho 0 (vô cực) bằng cách thay thế bằng 0

In [112]:
df_daily['Growth_Rate'] = df_daily['Growth_Rate'].replace([np.inf, -np.inf], 0).fillna(0)

## 6. LƯU FILE 1: DỮ LIỆU CHUỖI THỜI GIAN

In [113]:
# Chỉ giữ lại các cột biến động theo thời gian
cols_export_daily = ['Date_reported', 'Country_code', 'Country', 'WHO_region',
                     'New_cases', 'New_deaths', 'Cumulative_cases', 'Cumulative_deaths',
                     'New_cases_MA7', 'New_deaths_MA7', 'Growth_Rate'] # <--- Nhớ thêm Growth_Rate vào đây
# Thêm đuôi .gz và tham số compression='gzip'
df_daily[cols_export_daily].to_csv("01_clean_daily_timeseries.csv.gz", index=False, compression='gzip')

print(f"Đã lưu file nén 1: 01_clean_daily_timeseries.csv.gz")

Đã lưu file nén 1: 01_clean_daily_timeseries.csv.gz


# --- PHẦN 2: XỬ LÝ DỮ LIỆU DÂN SỐ & TỔNG HỢP QUỐC GIA ---
- Dành cho bạn Kiều Diễm làm Thống kê mô tả

## 1. Lấy danh sách quốc gia duy nhất từ dữ liệu gốc (để làm mốc)

- Lấy dòng dữ liệu mới nhất của mỗi nước để lấy số Cumulative cuối cùng

In [114]:
df_country = df_daily.sort_values('Date_reported').drop_duplicates('Country', keep='last').copy()

- Chọn các cột định danh và cột tổng kết

In [115]:
cols_static = ['Country_code', 'Country', 'WHO_region', 'Cumulative_cases', 'Cumulative_deaths']
df_country = df_country[cols_static].reset_index(drop=True)
df_country = df_country.rename(columns={'Cumulative_cases': 'Total_Cases', 'Cumulative_deaths': 'Total_Deaths'})

## 2. Xử lý chuẩn hóa tên nước & Mã ISO3

In [116]:
def alpha2_to_alpha3(code):
    try:
        return pycountry.countries.get(alpha_2=code).alpha_3
    except:
        return None
def get_country_code3_by_name(name):
    try:
        choices = [c.name for c in pycountry.countries]
        match = process.extractOne(name, choices)
        if match and match[1] > 80:
            return pycountry.countries.get(name=match[0]).alpha_3
    except:
        return None

- Tạo mã ISO3 cho df_country

In [117]:
df_country['Country_code3'] = df_country['Country_code'].apply(alpha2_to_alpha3)

- (Thêm logic fix lỗi tên nước và special_codes của bạn vào đây để điền nốt mã thiếu)

In [118]:
df_country['Country_clean'] = df_country['Country'].replace({
    'Cura�ao': 'Curacao',
    'Saint Martin (French part)': 'Saint Martin',
    'Sao Tome and Principe': 'Sao Tome and Principe',
    'British Virgin Islands': 'British Virgin Islands',
    'Turks and Caicos Islands': 'Turks and Caicos Islands',
    'Pitcairn': 'Pitcairn Islands',
    'Montenegro': 'Montenegro',
    'Gambia': 'Gambia',
    'Falkland Islands (Malvinas)': 'Falkland Islands',
    'United States Virgin Islands': 'United States Virgin Islands',
    'Bonaire, Sint Eustatius and Saba': 'Bonaire',
    'Bahamas': 'Bahamas',
    'Andorra': 'Andorra',
    'occupied Palestinian territory, including east Jerusalem': 'Palestine',
    'Sint Maarten (Dutch part)': 'Sint Maarten',
    'R�union': 'Reunion',
    'North Macedonia': 'Macedonia',
    'Saint Barthelemy': 'Saint Barthelemy',
    'Netherlands (Kingdom of the)': 'Netherlands',
    'Micronesia (Federated States of)': 'Micronesia',
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Czechia': 'Czech Republic',
    'Eswatini': 'Swaziland',
    'Cabo Verde': 'Cape Verde',
    'Brunei Darussalam': 'Brunei',
    'Timor-Leste': 'East Timor',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'Holy See': 'Vatican City',
    'Kosovo (in accordance with UN Security Council Resolution 1244 (1999))': 'Kosovo',
    'Côte d’Ivoire': 'Ivory Coast',
    'Türkiye': 'Turkey',
    'Viet Nam': 'Vietnam',
    "Lao People's Democratic Republic": 'Laos',
    'Iran (Islamic Republic of)': 'Iran',
    'Republic of Korea': 'South Korea',
    'Syrian Arab Republic': 'Syria',
    'Russian Federation': 'Russia',
    'Venezuela (Bolivarian Republic of)': 'Venezuela'
})

In [119]:
df_country['Country_code3'] = df_country.apply(
    lambda r: get_country_code3_by_name(r['Country_clean']) if pd.isna(r['Country_code3']) else r['Country_code3'],
    axis=1
)
special_codes = {
    'Taiwan': 'TWN',
    'Palestine': 'PSE',
    'Kosovo': 'XKX',
    'Namibia': 'NAM'
}

df_country['Country_code3'] = df_country.apply(
    lambda r: special_codes.get(r['Country_clean'], r['Country_code3']),
    axis=1
)

## 3. Đọc và ghép dữ liệu Dân số (Population)

In [120]:
url_pop = "https://raw.githubusercontent.com/datasets/population/master/data/population.csv"
pop = pd.read_csv(url_pop)
pop_latest = pop.sort_values('Year').drop_duplicates('Country Code', keep='last')
pop_latest = pop_latest.rename(columns={'Country Code': 'Country_code3', 'Value': 'Population'})[['Country_code3', 'Population']]

- Bổ sung dân số thủ công (Manual pop)

In [121]:
manual_pop = {
    'PCN': 50,
    'MSR': 4900,
    'BLM': 10000,
    'GUF': 300000,
    'MTQ': 370000,
    'BES': 26000,
    'VAT': 800,
    'GLP': 400000,
    'REU': 870000,
    'SHN': 6000,   # Saint Helena
    'MYT': 280000, # Mayotte
    'TKL': 1500,   # Tokelau
    'WLF': 12000,  # Wallis and Futuna
    'NIU': 1600,   # Niue
    'SPM': 6000,   # Saint Pierre and Miquelon
    'COK': 18000,  # Cook Islands
    'AIA': 15000,  # Anguilla
    'GGY': 67000,  # Guernsey
    'JEY': 108000, # Jersey
    'FLK': 3500,   # Falkland Islands

}
manual_df = pd.DataFrame(list(manual_pop.items()), columns=['Country_code3', 'Population'])
pop_latest = pd.concat([pop_latest, manual_df], ignore_index=True)

- MERGE: Ghép dân số vào bảng Quốc gia

In [122]:
df_country = df_country.merge(pop_latest, on='Country_code3', how='left')

- Loại bỏ các thực thể không phải quốc gia (Tàu thuyền)

In [123]:
df_country = df_country.dropna(subset=['Population'])

## 4. Tính các chỉ số thống kê tỷ lệ (Rates)

In [124]:
df_country['Cases_per_1M'] = df_country['Total_Cases'] / (df_country['Population'] / 1_000_000)
df_country['Deaths_per_1M'] = df_country['Total_Deaths'] / (df_country['Population'] / 1_000_000)
df_country['Fatality_Rate'] = (df_country['Total_Deaths'] / df_country['Total_Cases']) * 100


## 5. LƯU FILE 2: DỮ LIỆU TỔNG HỢP QUỐC GIA

In [125]:
df_country.to_csv("02_country_population_summary.csv.gz", index=False, compression='gzip')
print(f"Đã lưu file nén 2: 02_country_population_summary.csv.gz")

Đã lưu file nén 2: 02_country_population_summary.csv.gz


# Lọc 4 quốc gia: Việt nam, Ấn độ, Mỹ và Trung Quốc


In [11]:
import pandas as pd

# 1. Import dữ liệu đã làm sạch
df_daily = pd.read_csv("../data/processed/01_clean_daily_timeseries.csv.gz", compression='gzip')

# 2. Lọc 4 quốc gia
countries = ["Viet Nam", "China", "India", "United States of America"]
df_4 = df_daily[df_daily["Country"].isin(countries)].copy()
df_4 = df_4.sort_values(["Country", "Date_reported"])
 
df_4.to_csv("../data/processed/03_4_country_population_summary.csv.gz", index=False, compression='gzip')

print(df_4.head(20))


      Date_reported Country_code Country WHO_region  New_cases  New_deaths  \
87990    2020-01-04           CN   China        WPR        1.0         0.0   
87991    2020-01-05           CN   China        WPR        0.0         0.0   
87992    2020-01-06           CN   China        WPR        3.0         0.0   
87993    2020-01-07           CN   China        WPR        0.0         0.0   
87994    2020-01-08           CN   China        WPR        0.0         0.0   
87995    2020-01-09           CN   China        WPR        0.0         0.0   
87996    2020-01-10           CN   China        WPR        0.0         0.0   
87997    2020-01-11           CN   China        WPR        0.0         0.0   
87998    2020-01-12           CN   China        WPR       41.0         1.0   
87999    2020-01-13           CN   China        WPR        0.0         0.0   
88000    2020-01-14           CN   China        WPR        0.0         0.0   
88001    2020-01-15           CN   China        WPR        0.0  